<a href="https://colab.research.google.com/github/i-ganza007/PacMan_Formative/blob/main/MLPPOLICY_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y packaging numpy scipy torch tensorflow keras jax jaxlib pytensor numba opencv-python opencv-python-headless opencv-contrib-python bigframes datasets umap-learn gcsfs librosa statsmodels scikit-image arviz mizani imbalanced-learn yellowbrick sklearn-pandas treelite osqp pymc stumpy lightgbm hyperopt xgboost fastai cvxpy ale-py gymnasium stable-baselines3
!pip install -q --force-reinstall pip setuptools wheel

!apt-get update -qq && apt-get install -y -qq swig cmake libopenmpi-dev zlib1g-dev

!pip install -q \
    "gymnasium[box2d,atari,accept-rom-license]" \
    "stable-baselines3[extra]>=2.0.0" \
    "ale-py" \
    "torch==2.8.0"

!pip cache purge

Found existing installation: packaging 25.0
Uninstalling packaging-25.0:
  Successfully uninstalled packaging-25.0
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scipy 1.16.3
Uninstalling scipy-1.16.3:
  Successfully uninstalled scipy-1.16.3
Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Found existing installation: keras 3.10.0
Uninstalling keras-3.10.0:
  Successfully uninstalled keras-3.10.0
Found existing installation: jax 0.7.2
Uninstalling jax-0.7.2:
  Successfully uninstalled jax-0.7.2
Found existing installation: jaxlib 0.7.2
Uninstalling jaxlib-0.7.2:
  Successfully uninstalled jaxlib-0.7.2
Found existing installation: pytensor 2.35.1
Uninstalling pytensor-2.35.1:
  Successfully uninstalled py

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')

import ale_py
import gymnasium as gym
gym.register_envs(ale_py)
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd
import numpy as np
import random
import torch
import gc

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [3]:
gc.collect()
torch.cuda.empty_cache()

In [4]:
CONFIG_ID = 2

configs = [
    {"lr": 1e-4, "gamma": 0.99, "batch": 32,  "eps_start": 1.0, "eps_end": 0.1,  "eps_decay": 0.1},
    {"lr": 5e-5, "gamma": 0.99, "batch": 64,  "eps_start": 1.0, "eps_end": 0.05, "eps_decay": 0.2},
    {"lr": 2e-4, "gamma": 0.98, "batch": 32,  "eps_start": 1.0, "eps_end": 0.1,  "eps_decay": 0.15},
    {"lr": 1e-4, "gamma": 0.95, "batch": 64,  "eps_start": 1.0, "eps_end": 0.2,  "eps_decay": 0.3},
    {"lr": 7e-5, "gamma": 0.99, "batch": 32,  "eps_start": 1.0, "eps_end": 0.01, "eps_decay": 0.05},
    {"lr": 1e-4, "gamma": 0.97, "batch": 128, "eps_start": 1.0, "eps_end": 0.1,  "eps_decay": 0.5},
    {"lr": 5e-5, "gamma": 0.995,"batch": 16,  "eps_start": 1.0, "eps_end": 0.05, "eps_decay": 0.2},
    {"lr": 3e-4, "gamma": 0.99, "batch": 32,  "eps_start": 1.0, "eps_end": 0.1,  "eps_decay": 0.1},
    {"lr": 1e-5, "gamma": 0.99, "batch": 128, "eps_start": 1.0, "eps_end": 0.1,  "eps_decay": 0.3},
    {"lr": 1e-4, "gamma": 0.97, "batch": 64,  "eps_start": 1.0, "eps_end": 0.01, "eps_decay": 0.4},
]


cfg = configs[CONFIG_ID - 1]


print(f"Policy: MlpPolicy")
print(f"Hyperparameters:")
print(f"  • Learning Rate (lr): {cfg['lr']}")
print(f"  • Gamma (γ): {cfg['gamma']}")
print(f"  • Batch Size: {cfg['batch']}")
print(f"  • Epsilon Start: {cfg['eps_start']}")
print(f"  • Epsilon End: {cfg['eps_end']}")
print(f"  • Epsilon Decay: {cfg['eps_decay']}")

Policy: MlpPolicy
Hyperparameters:
  • Learning Rate (lr): 5e-05
  • Gamma (γ): 0.99
  • Batch Size: 64
  • Epsilon Start: 1.0
  • Epsilon End: 0.05
  • Epsilon Decay: 0.2


In [5]:
env_test = gym.make("ALE/SpaceInvaders-v5")
random_scores = []

for episode in range(1, 21):
    obs, _ = env_test.reset()
    done = False
    truncated = False
    score = 0
    while not (done or truncated):
        action = random.choice([0, 1, 2, 3, 4, 5])
        obs, reward, done, truncated, _ = env_test.step(action)
        score += reward
    random_scores.append(score)
    if episode % 5 == 0:
        print(f"  Episode {episode}: {score}")

avg_random_score = np.mean(random_scores)
print(f"\nRandom Agent Average Score: {avg_random_score:.2f}")
env_test.close()

  Episode 5: 155.0
  Episode 10: 80.0
  Episode 15: 155.0
  Episode 20: 390.0

Random Agent Average Score: 133.00


In [6]:
env = gym.make("ALE/SpaceInvaders-v5", frameskip=4)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4)

In [7]:
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
log_dir = "./logs/spaceinvaders/"
os.makedirs(log_dir, exist_ok=True)
model = DQN(
    "MlpPolicy",
    env,
    learning_rate=cfg["lr"],
    gamma=cfg["gamma"],
    batch_size=cfg["batch"],
    buffer_size=10000,
    learning_starts=5000,
    exploration_initial_eps=cfg["eps_start"],
    exploration_final_eps=cfg["eps_end"],
    exploration_fraction=cfg["eps_decay"],
    target_update_interval=1000,
    train_freq=4,
    gradient_steps=1,
    tensorboard_log=log_dir,
    verbose=1,
    device="cuda"
)

print("Model created successfully!")
print(f"   Buffer size: {model.buffer_size:,}")
print(f"   Learning starts at: {model.learning_starts:,} steps")
print(f"   Policy:  MlpPolicy ( MlpPolicy)")

print(f"\nInitial GPU memory: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

Using cuda device
Wrapping the env in a VecTransposeImage.
Model created successfully!
   Buffer size: 10,000
   Learning starts at: 5,000 steps
   Policy:  MlpPolicy ( MlpPolicy)

Initial GPU memory: 0.19 GB


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [8]:

checkpoint_callback = CheckpointCallback(
    save_freq=50_000,
    save_path="./checkpoints/",
    name_prefix="dqn_spaceinvaders"
)

eval_callback = EvalCallback(
    env,
    best_model_save_path="./best_model/",
    log_path="./eval_logs/",
    eval_freq=10_000,
    deterministic=True,
    render=False
)

callback_list = [checkpoint_callback, eval_callback]

try:
    model.learn(
        total_timesteps=500_000,
        tb_log_name=f"SpaceInvaders_Config{CONFIG_ID}MLPolicy",
        log_interval=50,
        progress_bar=True,
        callback=callback_list
    )

    print("\nTraining completed successfully!")
    print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**3:.2f} GB")

except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print("\nOUT OF MEMORY ERROR")
        print(f"Current settings: buffer={model.buffer_size}, batch={cfg['batch']}")
        print("\nEMERGENCY FIX:")
        print("   1. Change buffer_size=5000")
        print("   2. Change batch_size=4")
        print("   3. Or use device='cpu'")
    raise e

 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500,000/500,000  [ 2:30:08 < 0:00:00 , ? it/s ]


Training completed successfully!
Max GPU memory used: 0.74 GB


In [9]:
mean_reward, std_reward = evaluate_policy(
    model,
    env,
    n_eval_episodes=10,
    deterministic=True
)

print(f"\nResults over 10 episodes:")
print(f"   Mean Reward: {mean_reward:.2f} ± {std_reward:.2f}")
print(f"   Random Baseline: {avg_random_score:.2f}")
print(f"   Improvement: {mean_reward - avg_random_score:.2f}")


Results over 10 episodes:
   Mean Reward: 217.50 ± 75.64
   Random Baseline: 133.00
   Improvement: 84.50


In [10]:
model_name = "dqn_model_MLP_2"
model.save(model_name)

backup_name = f"dqn_spaceinvaders_exp_MLP{CONFIG_ID}"
model.save(backup_name)

print(f"\nModels saved:")
print(f"   • {model_name}.zip (required submission)")
print(f"   • {backup_name}.zip (backup with config ID)")


Models saved:
   • dqn_model_MLP_2.zip (required submission)
   • dqn_spaceinvaders_exp_MLP2.zip (backup with config ID)


### Hyperparameter Configurations

| Config ID | Learning Rate (lr) | Gamma (γ) | Batch Size | Epsilon Start | Epsilon End | Epsilon Decay |
|---|---|---|---|---|---|---|
| 1 | 1e-4 | 0.99 | 32 | 1.0 | 0.1 | 0.1 |
| 2 | 5e-5 | 0.99 | 64 | 1.0 | 0.05 | 0.2 |
| 3 | 2e-4 | 0.98 | 32 | 1.0 | 0.1 | 0.15 |
| 4 | 1e-4 | 0.95 | 64 | 1.0 | 0.2 | 0.3 |
| 5 | 7e-5 | 0.99 | 32 | 1.0 | 0.01 | 0.05 |
| 6 | 1e-4 | 0.97 | 128 | 1.0 | 0.1 | 0.5 |
| 7 | 5e-5 | 0.995 | 16 | 1.0 | 0.05 | 0.2 |
| 8 | 3e-4 | 0.99 | 32 | 1.0 | 0.1 | 0.1 |
| 9 | 1e-5 | 0.99 | 128 | 1.0 | 0.1 | 0.3 |
| 10 | 1e-4 | 0.97 | 64 | 1.0 | 0.01 | 0.4 |


In [11]:
print(configs)

[{'lr': 0.0001, 'gamma': 0.99, 'batch': 32, 'eps_start': 1.0, 'eps_end': 0.1, 'eps_decay': 0.1}, {'lr': 5e-05, 'gamma': 0.99, 'batch': 64, 'eps_start': 1.0, 'eps_end': 0.05, 'eps_decay': 0.2}, {'lr': 0.0002, 'gamma': 0.98, 'batch': 32, 'eps_start': 1.0, 'eps_end': 0.1, 'eps_decay': 0.15}, {'lr': 0.0001, 'gamma': 0.95, 'batch': 64, 'eps_start': 1.0, 'eps_end': 0.2, 'eps_decay': 0.3}, {'lr': 7e-05, 'gamma': 0.99, 'batch': 32, 'eps_start': 1.0, 'eps_end': 0.01, 'eps_decay': 0.05}, {'lr': 0.0001, 'gamma': 0.97, 'batch': 128, 'eps_start': 1.0, 'eps_end': 0.1, 'eps_decay': 0.5}, {'lr': 5e-05, 'gamma': 0.995, 'batch': 16, 'eps_start': 1.0, 'eps_end': 0.05, 'eps_decay': 0.2}, {'lr': 0.0003, 'gamma': 0.99, 'batch': 32, 'eps_start': 1.0, 'eps_end': 0.1, 'eps_decay': 0.1}, {'lr': 1e-05, 'gamma': 0.99, 'batch': 128, 'eps_start': 1.0, 'eps_end': 0.1, 'eps_decay': 0.3}, {'lr': 0.0001, 'gamma': 0.97, 'batch': 64, 'eps_start': 1.0, 'eps_end': 0.01, 'eps_decay': 0.4}]
